In [19]:
import pandas as pd
import numpy as np
import re
import Levenshtein

df = pd.read_csv(r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Sun.csv')
#df.info()
#df

In [20]:
# Convert the column to dtype 'datatime'
df['Submitted at'] = pd.to_datetime(df['Submitted at'])

# Convert the column to dtype 'object' to handle strings properly
df['surveyid'] = df['surveyid'].astype('object')

# Replace 'Untitled multiple choice field' with 'More Info' in all column names
df.columns = df.columns.str.replace(r'Untitled multiple choice field', 'More Info')
#df

# Remove the 'Submitted at' column...
column = df.pop('Submitted at')

# Insert 'Submitted at' column at index position 6
df.insert(6, 'Submitted at', column)

# Insert two new columns that we need later
df.insert(loc=6, column='Response_Count', value='0')
df.insert(loc=0, column='Is_Duplicate', value=False)

df.info()
#df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 42 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   Is_Duplicate                                                                  217 non-null    bool          
 1   Submission ID                                                                 217 non-null    object        
 2   Respondent ID                                                                 217 non-null    object        
 3   can_id                                                                        213 non-null    object        
 4   surveyid                                                                      0 non-null      object        
 5   Name                                                                          176 non-null  

In [21]:
# Drop all rows when 'can_id' is NaN

df.dropna(subset=['can_id'], inplace=True)

#df

In [22]:
# Fill 'surveyid' with the specified string
# How do we pass this string into the DataFrame script?

df['surveyid'] = df['surveyid'].fillna('Week 05 Sun')

#df

In [23]:
# Email validation pattern
email_pattern = re.compile(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")

# Common email domains
common_domains = {
    "350seattle.org": "350seattle.org",
    "aol.com": "aol.com",
    "comcast.net": "comcast.net",
    "duck.com": "duck.com",
    "fastmail.com": "fastmail.com",
    "frontier.com": "frontier.com",
    "gmail.com": "gmail.com",
    "hotmail.com": "hotmail.com",
    "icloud.com": "icloud.com",
    "mac.com": "mac.com",
    "me.com": "me.com",
    "msn.com": "msn.com",
    "outlook.com": "outlook.com",
    "pobox.com": "pobox.com",
    "proton.me": "proton.me",
    "protonmail.com": "protonmail.com",
    "uw.edu": "uw.edu",
    "yahoo.com": "yahoo.com"
}

# Common TLD misspellings
tld_corrections = {
    'com': ['copm', 'co', 'cpm', 'comm', 'copm'],
    'org': ['ogr', 'orgg', 'or'],
    'net': ['nett', 'ne', 'nte'],
    'gov': ['gvo', 'govv', 'go'],
    'edu': ['ed', 'edd', 'eud']
}

# Function to correct common TLD misspellings
def correct_tld(domain):
    tld = domain.split('.')[-1]
    base_domain = '.'.join(domain.split('.')[:-1])
    for correct_tld, misspellings in tld_corrections.items():
        if tld in misspellings:
            return f"{base_domain}.{correct_tld}"
    return domain

# Function to validate and correct email addresses
def validate_and_correct_email(email):
    if isinstance(email, str) and email_pattern.match(email):
        domain = email.split('@')[1]
        corrected_domain = correct_tld(domain)
        if corrected_domain in common_domains:
            return email.split('@')[0] + '@' + corrected_domain
        else:
            # Find the closest match if the domain is incorrect
            corrected_domain = min(common_domains.keys(), key=lambda k: Levenshtein.distance(corrected_domain, k))
            return email.split('@')[0] + '@' + corrected_domain
    else:
        return None

# Convert all email values to strings, replacing NaN with empty string
df['Email'] = df['Email'].astype(str).fillna('')

# Validate and correct email addresses
df['Email'] = df['Email'].apply(validate_and_correct_email)

#df.info()
#df

In [24]:
# Setup df to compare and fill empty 'Email' and 'Name' values based on 'Respondent ID'

# Identify duplicates in 'Respondent ID' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['Respondent ID'].duplicated(keep=False)

# Sort by 'Respondent ID' and reset index
sorted_df = df.sort_values(by=['Respondent ID'], ascending=[True])
sorted_df = sorted_df.reset_index(drop=True) # Reset index

df = sorted_df
#df

In [25]:
# Identify the duplicates for 'Respondent ID'
duplicate_respondents = df[df.duplicated('Respondent ID', keep=False)]

# Group by 'Respondent ID' to handle each set of duplicates
grouped = duplicate_respondents.groupby('Respondent ID')

# Function to fill empty values in the 'Email' and 'Name' columns for each group
def fill_columns(group):
    group = group.copy()  # Avoid SettingWithCopyWarning
    
    # Find non-NaN values if they exist
    non_nan_email = group['Email'].dropna().iloc[0] if not group['Email'].dropna().empty else None
    non_nan_name = group['Name'].dropna().iloc[0] if not group['Name'].dropna().empty else None
    
    # Fill NaN values with non-NaN values found within the group
    if non_nan_email:
        group['Email'] = group['Email'].fillna(non_nan_email)
    if non_nan_name:
        group['Name'] = group['Name'].fillna(non_nan_name)
    
    return group

# Apply the function to each group and reset the index
RespondentID_groups = grouped.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)

# Combine the manipulated groups with the original DataFrame
# First, drop the rows that were manipulated from the original DataFrame
df_non_duplicates = df[~df['Respondent ID'].isin(RespondentID_groups['Respondent ID'])]

# Concatenate the manipulated groups with the non-duplicate rows
final_df = pd.concat([df_non_duplicates, RespondentID_groups]).sort_values(by='Respondent ID').reset_index(drop=True)

df = final_df
#df

C:\Users\peter\AppData\Local\Temp\ipykernel_12880\1744984796.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  RespondentID_groups = grouped.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)


In [26]:
# Setup df to compare and fill empty 'Email' and 'Name' values based on 'can_id'

# Identify duplicates in 'can_id' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['can_id'].duplicated(keep=False)

# Sort by 'can_id' and reset index
sorted_df = df.sort_values(by=['can_id'], ascending=[True])
sorted_df = sorted_df.reset_index(drop=True) # Reset index

df = sorted_df
#df

In [27]:
# Identify the duplicates for 'can_id'
duplicate_candidates = df[df.duplicated('can_id', keep=False)]

# Group by 'can_id' to handle each set of duplicates
can_id_groups = duplicate_candidates.groupby('can_id')

# Function to fill 'Email' and 'Name' columns for each group
def fill_columns(group):
    group = group.copy()  # Avoid SettingWithCopyWarning
    
    # Find non-NaN values if they exist
    non_nan_email = group['Email'].dropna().unique()
    non_nan_name = group['Name'].dropna().unique()
    
    # Check for the unique non-NaN values for 'Email' or 'Name'
    if len(non_nan_email) == 1 and len(non_nan_name) == 1:
        group['Email'] = group['Email'].fillna(non_nan_email[0])
        group['Name'] = group['Name'].fillna(non_nan_name[0])

    elif len(non_nan_email) == 1 and len(non_nan_name) == 0:
        group['Email'] = group['Email'].fillna(non_nan_email[0])

    elif len(non_nan_email) == 0 and len(non_nan_name) == 1:
        group['Name'] = group['Name'].fillna(non_nan_name[0])
    
    return group

# Apply the function to each group and reset the index
can_id_groups_filled = can_id_groups.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)

# Combine the filled groups with the original DataFrame
# First, drop the rows that were manipulated from the original DataFrame
df_non_duplicates = df[~df['can_id'].isin(can_id_groups_filled['can_id'])]

# Concatenate the filled groups with the non-duplicate rows
final_df = pd.concat([df_non_duplicates, can_id_groups_filled]).sort_values(by='can_id').reset_index(drop=True)

df = final_df
#df

C:\Users\peter\AppData\Local\Temp\ipykernel_12880\3683480017.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  can_id_groups_filled = can_id_groups.apply(lambda x: fill_columns(x.reset_index(drop=True))).reset_index(drop=True)


In [28]:
# Use fillna to replace the remaining NaN 'Email' values with 'can_id'
df['Email'] = df['Email'].fillna(df['can_id'])
#df

In [29]:
# Setup df for comparing and updating empty column values for duplicate rows

# Identify duplicates in 'can_id' with 'True' in 'Is_Duplicate' column
df['Is_Duplicate'] = df['Email'].duplicated(keep=False)

# Sort by 'Email' and 'Submitted at' in ascending order
sorted_df = df.sort_values(by=['Email', 'Submitted at'], ascending=[True, True])

sorted_df = sorted_df.reset_index(drop=True) # Reset index
df = sorted_df
#df

In [30]:
# Compare and update empty column values for duplicate rows so that we retain all action taker's entered values
# Then drop the rows that are unneeded duplicates

indices_to_drop = []

for x in range(len(df) - 1):  # Iterate until the second last row
	if (df.loc[x, 'Email'] == df.loc[x + 1, 'Email']):

		# Iterate over each column starting from the 10th column (index 9 since indexing is zero-based)
		for col in df.columns[9:]:
			if pd.isna(df.at[x+1, col]):  # Check if the next row's value is empty
				df.at[x+1, col] = df.at[x, col]  # Set the value to the current row's value
	
		indices_to_drop.append(x)
	
df = df.drop(indices_to_drop)

df = df.reset_index(drop=True) # Reset index

#df

In [31]:
# Starting in the 9th column, count the number of non-NaN values in each row and store in 'Response_Count'
# However, do NOT count values in the UX columns
# 'Response_Count' quantifies each action taker's level of engagement at the survey level
# NOTE: We should NOT include rows of data in certain analyses if 'Response_Count' == 0

# List of strings to exclude columns
exclude_strings = ['More Info', 'Additional ideas for your comment', 'Email link not work for you?']

# Create a mask for columns to exclude
exclude_mask = df.columns.str.contains('|'.join(exclude_strings))

# Create a mask for columns to include (starting from the 10th column, index 9)
include_mask = ~exclude_mask & (np.arange(len(df.columns)) >= 9)

# Count non-NaN values in each row for the included columns and store in 'Response_Count'
df['Response_Count'] = df.loc[:, include_mask].notna().sum(axis=1)

#df

In [32]:

# Cleanup of unneeded column
df = df.drop('Is_Duplicate', axis=1)

# Define file path
# csv_path = r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Sun_cleaned.csv'

# Write DataFrame to CSV file
# df.to_csv(csv_path, index=False)

In [33]:
# Determine the attribute columns starting from the 7th column (index 6)
attribute_columns = df.columns[6:]

# Ensure that we have the correct columns
print("Attribute columns:", attribute_columns)

# Unpivot the DataFrame
unpivoted_df = pd.melt(df, id_vars=df.columns[:6], value_vars=attribute_columns, var_name='Attribute', value_name='Value')

# Convert 'Attribute' to a categorical type to maintain the order dynamically
unpivoted_df['Attribute'] = pd.Categorical(unpivoted_df['Attribute'], categories=attribute_columns, ordered=True)

# Sort by 'ID' and the categorical 'Attribute' to maintain the sequence and reset index
unpivoted_df = unpivoted_df.sort_values(['Email', 'Attribute']).reset_index(drop=True)

Attribute columns: Index(['Response_Count', 'Submitted at', 'More Info (1)',
       'Additional ideas for your comment (1)',
       'Did you send a comment to your Senator about SB 5955?',
       'Run into issues with SB 5955 or need to elaborate?', 'More Info (2)',
       'Additional ideas for your comment (2)',
       'Did you send a comment to your two Representatives about HB 2156?',
       'Run into issues with HB 2156 or need to elaborate?', 'More Info (3)',
       'Additional ideas for your comment (3)',
       'Did you send a comment to your two Representatives about HB 1185?',
       'Run into issues with HB 1185 or need to elaborate?', 'More Info (4)',
       'Email link not work for you? (1)',
       'Did you email the 58 members of the House about HB 1445??',
       'Run into issues with HB 1445? or need to elaborate?', 'More Info (5)',
       'Additional ideas for your comment (4)',
       'Did you send a comment to your two Representatives about HB 1513?',
       'Run int

In [34]:
# Create a new 'Key ID' column by concatenating 'Submission ID' + a number in the format of 01, 02, 03, etc.

# Create the 'Key ID' column with the formatted string
unpivoted_df.insert(0, 'Key ID', unpivoted_df.groupby('Submission ID').cumcount() + 1)

# Concatenate 'Submission ID' to the 'Key ID' two-digit number with leading zeros
unpivoted_df['Key ID'] = unpivoted_df['Submission ID'] + unpivoted_df['Key ID'].apply(lambda x: f"{x:02}")

# Display the updated DataFrame
# print(unpivoted_df)


In [35]:
# Initialize an empty list to store the action numbers
action_numbers = []

# Iterate over each group in the DataFrame based on 'Submission ID'
for _, group in unpivoted_df.groupby('Submission ID'):
    # Initialize action number counter for each group
    action_number = 1
    
    # Iterate through the 'Attribute' column in the group
    for attribute in group['Attribute']:
        # Check if the current attribute contains 'Response_Count' or 'Submitted at'
        if 'Response_Count' in attribute or 'Submitted at' in attribute:
            # Append None to action_numbers if condition is met
            action_numbers.append(None)
        else:
            # Append the current action number to the list
            action_numbers.append(action_number)
            
            # Check if the current attribute marks the end of a subgroup
            if 'Run into issues with' in attribute:
                # Increment the action number for the next subgroup
                action_number += 1

# Assign the list of action numbers to the new 'Action Number' column in the DataFrame
unpivoted_df['Action Number'] = action_numbers

In [36]:
# Define file path
csv_path = r'D:\Users\peter\PythonProjects\PandasTest\data\Week 05 Sun_normalized.csv'

# Write DataFrame to CSV file
unpivoted_df.to_csv(csv_path, index=False)
